In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F 

import utils.tavr_torch as tavr_torch
from utils.tavr_torch import TAVR_3_Frame, TAVR_1_Frame, TAVR_Sequence, tavr_dataloader
from utils.visualization import display_grid, z_stretch, visualize_frame, set_figsize, get_central_slices
from utils.loss_functions import batch_l2_loss
from utils.run_model import train, test, save, load, get_loss_history
from Models.basic_models import average_model, post_process
from Models.two_layer_resnet import two_layer_resnet

set_figsize(20, 15)

Training directory found, 36 series
Validation directory found, 6 series
Testing directory found, 10 series


In [4]:
USE_GPU = True
dtype = torch.float32 # we will be using float throughout this tutorial
if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print('using device:', device)

# "Pixl", "Slice", or "None"
preproc_type = "pixel"

validation = TAVR_3_Frame("__valid", preproc=preproc_type)
val_loader = tavr_dataloader(validation, batch_size=4, shuffle=True, num_workers=2)
training = TAVR_3_Frame("__train", preproc=preproc_type)
train_loader = tavr_dataloader(training,batch_size=8, shuffle=True, num_workers=2)


ave_model = average_model()
model = two_layer_resnet()
post_proc = post_process(kind=preproc_type).to(device=device)
loss_fn = batch_l2_loss()

# CHANGE TO NAME OF JUPYTER NOTEBOOK
model_name = "Model 7 (two_layer_resnet) Run 1 [SGD]"

using device: cpu


In [5]:
learning_rate = 3e-3
reg = 1e-7

optimizer = optim.SGD(model.parameters(), lr=learning_rate, weight_decay=reg)

In [6]:
# Run cell to load model
LOAD = False
iteration_num = -1

if LOAD:
    
    load(model_name, iteration_num, model, optimizer)
    loss_history = get_loss_history(model_name)
    model.to(device=device)
else:
    loss_history = None

In [7]:
train(model, post_proc, optimizer, train_loader, val_loader, loss_fn, device, 
         model_name, loss_history, epochs=3, print_every=30, print_level=4, lr_decay=1)


Iteration 0, loss = 1.1833, corrected loss = 550.5978
Validation loss 509.7789 over 81 frames
model saved to model_checkpoints/Model 7 (two_layer_resnet) Run 1 [SGD]/Model 7 (two_layer_resnet) Run 1 [SGD]-0
conv_a1.weight,   	norm: 4.0602e+00, 	update norm: 7.9647e-03 	Update/norm: 1.9616e-03
conv_a1.bias,   	norm: 2.3213e-01, 	update norm: 1.8233e-03 	Update/norm: 7.8547e-03
conv_b1.weight,   	norm: 3.8010e+00, 	update norm: 1.0327e-02 	Update/norm: 2.7168e-03
conv_b1.bias,   	norm: 3.4081e-01, 	update norm: 2.0960e-03 	Update/norm: 6.1500e-03
final.weight,   	norm: 1.9068e+00, 	update norm: 8.9602e-03 	Update/norm: 4.6991e-03
final.bias,   	norm: 1.6076e-01, 	update norm: 2.6465e-03 	Update/norm: 1.6462e-02

... 0.9694... 0.6587... 0.5389
Iter 10... 0.3828... 0.3581... 0.3485
Iter 20... 0.3705... 0.3388... 0.3508
Iteration 30, loss = 0.3076, corrected loss = 143.1405
Validation loss 154.7427 over 81 frames
model saved to model_checkpoints/Model 7 (two_layer_resnet) Run 1 [SGD]/Model

In [ ]:
# One last test + visualize results on 1 validation sequence
val_seq = TAVR_Sequence("__valid")
test(model, val_loader, loss_fn, device)
test(ave_model, val_loader, loss_fn, device)

with torch.no_grad():
    model.eval()
    ave_frames = [val_seq[0][0]]
    for i in range(2,9,2):
        ave_frame = model((val_seq[0][i-2][None,:], val_seq[0][i][None,:]))
        ave_frames += [ave_frame[0][0],  val_seq[0][i]]
    ave_frames += [val_seq[0][9]]
    ave_frames_slices = []
    for f in ave_frames:
        ave_frames_slices += get_central_slices(f)
    set_figsize(6,20)
    display_grid(10, 3, ave_frames_slices)

In [ ]:
optimizer.param_groups[0]['lr']

In [ ]:
for p in model.parameters():
    print(p.norm())